In [1]:
n_rays = 50
case = "_Ex_7_1"

In [2]:
params = {
    "initialization": {
        "x_init": [0.5],
        "y_init": [0.8, 0.5]
    },
    "exact":{
        "learning_rate": 0.001,
        "max_iters": 20000,
        "tol": 1e-5
    },
    "phase1": {
        "gamma": 12,
        "expo_gamma": 0.01,
        "max_iter": 10000,
        "tol": 1e-4
    },
    "phase2": {
        "expo_alpha": 0.45,
        "expo_beta": 0.4,
        "expo_lambda": 0.55,
        "init_params": 1.0,
        "max_iter": 2000,
        "mu": 1e-4,
        "verbose": False,
        "expo_gamma": 0.5
    }
}


In [3]:
import autograd.numpy as np
import torch

In [4]:
def set_seed(seed=702):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed(702)

# Gen rays

In [5]:
import math

In [6]:
def generate_chebyshev_rays(num_rays=10):
    rays = []
    angles = np.linspace(0, np.pi/2, num_rays)
    for theta in angles:
        r1, r2 = math.cos(theta), math.sin(theta)
        if abs(r1) < 1e-9: r1 = 0.0
        if abs(r2) < 1e-9: r2 = 0.0
        rays.append([r1, r2])       
    return np.array(rays)

In [7]:
test_rays = generate_chebyshev_rays(n_rays)

# Giải chính xác bằng phương pháp chiếu

## Bài toán

In [8]:
# ==========================================
# HÀM MỤC TIÊU
# ==========================================
def f(x):    return np.array([x[0],(x[0] - 1)**2])
def f1(x):    return x[0]
def f2(x):    return (x[0] - 1)**2
def jacobian_f(x):
    return np.array([[1.0],[2.0 * (x[0] - 1)]])
# --- Hàm mục tiêu Weighted Chebyshev ---
def S_weighted_chebyshev(x, r, z):
    vals = f(x)
    weighted_diffs = r * np.abs(vals - z)
    return np.max(weighted_diffs)
# --- Tính Subgradient ---
def get_subgradient(x, r, z):
    """
    Tính đạo hàm của hàm max(r_i * |f_i - z_i|)
    """
    vals = f(x)
    diffs = vals - z
    weighted_abs_diffs = r * np.abs(diffs)
    k = np.argmax(weighted_abs_diffs)
    J = jacobian_f(x)
    gradient_fk = J[k] 
    sign = 1.0 if diffs[k] >= 0 else -1.0
    grad = r[k] * sign * gradient_fk
    return grad
# ==========================================
# CÁC RÀNG BUỘC & PHÉP CHIẾU
# ==========================================
# Ràng buộc tập X
def c1(x): return x[0]
def c2(x): return -x[0] + 1
# Ràng buộc tập Q
def q1(y): return 0.2**2 - (y[0] - 0.4)**2 - (y[1] - 0.4)**2
def q_plus(y):
    center = .4
    radius_sq = 0.2**2  
    dx = np.maximum(0, y[0] - center)
    dy = np.maximum(0, y[1] - center)
    return radius_sq - (dx**2 + dy**2)
dim_x = 1
dim_y = 2

## Phép chiếu lên X'

In [9]:
from Problem import Problem
from project import Projection
from scipy.optimize import minimize

In [10]:
cons_C = ({'type': 'ineq', 'fun' : c1,},
          {'type': 'ineq', 'fun' : c2,})
cons_Q = ({'type': 'ineq', 'fun' : q1,},)
cons_Qplus = ({'type': 'ineq', 'fun': q_plus},)
# Setup Projections
proj_C_handler = Projection(cons=cons_C, bounds=None, dim=dim_x, proj_type='euclid')
proj_Q_handler = Projection(cons=cons_Q, bounds=None, dim=dim_y, proj_type='qplus')
# Setup Problem
prob = Problem(
    f=[f], jac_f=[jacobian_f],
    dim_x=dim_x, dim_y=dim_y,
    proj_C=proj_C_handler.project,
    proj_Qplus=proj_Q_handler.project
)

In [11]:
def project_step(x_temp, y_temp, dim_x, dim_y):
    """
    Chiếu điểm (x, y) vào tập:
    { (x, y) | x in X, y in Q, f(x) <= y }
    """
    z_start = np.concatenate([x_temp, y_temp])
    def proj_obj(z): 
        return 0.5 * np.sum((z - z_start)**2)
    def proj_jac(z): 
        return z - z_start
    cons = [
        # Ràng buộc x in X
        {'type': 'ineq', 'fun': lambda z: c1(z[:dim_x])},
        {'type': 'ineq', 'fun': lambda z: c2(z[:dim_x])},
        # Ràng buộc y in Q
        {'type': 'ineq', 'fun': lambda z: q1(z[dim_x:])},
        # Ràng buộc f(x) <= y  <=>  y - f(x) >= 0
        {'type': 'ineq', 'fun': lambda z: z[dim_x:] - f(z[:dim_x])}
    ]

    res = minimize(
        fun=proj_obj, x0=z_start, jac=proj_jac,
        constraints=cons, method='SLSQP',
        options={'disp': False}
    )
    
    if not res.success:
        return x_temp, y_temp
    return res.x[:dim_x], res.x[dim_x:]

## Core

In [12]:
def solve_weighted_chebyshev(r_pref, z_ref, params , dim_x=1, dim_y=2):
    x = params["initialization"]["x_init"]    
    y = params["initialization"]["y_init"]
    learning_rate = params["exact"]["learning_rate"]
    max_iters = params["exact"]["max_iters"]
    tol = params["exact"]["tol"]
    prev_obj_val = float('inf')
    converge = False
    
    print(f"r={r_pref}")
    print(f"{'Iter':<5} | {'x':<10} | {'y vector':<15} | {'Objective':<10} | {'Active Idx'}")
    for k in range(max_iters):
        # --- BƯỚC 1: Tính Subgradient ---
        grad_x = get_subgradient(x, r_pref, z_ref)
        
        # --- BƯỚC 2: Cập nhật biến x ---
        x_temp = x - learning_rate * grad_x
        y_temp = y 
        
        # --- BƯỚC 3: Phép chiếu (Projection) ---
        x_next, y_next = project_step(x_temp, y_temp, dim_x, dim_y)
        
        # --- Tính giá trị hàm mục tiêu hiện tại ---
        obj_val = S_weighted_chebyshev(x_next, r_pref, z_ref)
        
        # --- Logging ---
        diffs = r_pref * np.abs(f(x_next) - z_ref)
        active_idx = np.argmax(diffs)
        
        if k % 2000 == 0:
            print(f"{k:<5} | {x_next[0]:.4f}     | [{y_next[0]:.2f}, {y_next[1]:.2f}] | {obj_val:.6f}     | f_{active_idx+1}")

        # --- BƯỚC 4: Kiểm tra điều kiện dừng ---
        loss_change = abs(prev_obj_val - obj_val)
        
        if loss_change < tol:
            print(f"--> Hội tụ tại bước {k}!\n")
            converge = True
            break
            
        prev_obj_val = obj_val
        x = x_next
        y = y_next

    return x, y, converge

## Tìm điểm tham chiếu

In [13]:
from Phase1 import solve_CQ_feasible
from Phase2_1_obj import optim_Universal

In [14]:
def relevant_point(prob, params):
    x_feasible, _, _, _ = solve_CQ_feasible(
        prob.objective_func, prob.jacobian, prob.proj_C, prob.proj_Qplus,
        x0=params['initialization']['x_init'],
        gamma=params['phase1']['gamma'] , max_iter=params['phase1']['max_iter']
    )

    z_vals = []
    nadir_vals = []

    for dim in range(2):
        # Tìm MIN
        x_min, _ = optim_Universal(prob, x_feasible, target_dim=dim, mode="min", max_iter=500, mu=0.001)
        val_min = prob.objective_func(x_min)[dim]
        z_vals.append(val_min)

        # Tìm MAX 
        x_max, _ = optim_Universal(prob, x_feasible, target_dim=dim, mode="max", max_iter=500, mu=0.001)
        val_max = prob.objective_func(x_max)[dim]
        nadir_vals.append(val_max)
        print(f"   Dim {dim}: Min={val_min:.4f}, Max={val_max:.4f}")
    z_star = np.array(z_vals)
    ref_point = np.array(nadir_vals) + np.abs(np.array(nadir_vals)) * 0.1 + 0.5 
    print(f"-> Z* (Ideal): {z_star}")
    print(f"-> Ref Point (HV): {ref_point}")
    print(f"-> x (Feasible): {x_feasible}")
    return z_star, ref_point, x_feasible

## Giải chính xác

In [15]:
z_star, ref_point, x_feasible = relevant_point(prob, params)

Khởi tạo: x0: [0.5]
Chiếu lên C được: x: [0.5]


  0%|          | 0/10000 [00:00<?, ?it/s]


Hội tụ tại vòng lặp 0
+---+-------+---------+--------------+--------------+----------+----------+
| k | x_new | gamma_k | y            | z_proj       |   e_x    |   e_f    |
+---+-------+---------+--------------+--------------+----------+----------+
| 0 | [0.5] | 12.0000 | [0.5 , 0.25] | [0.5 , 0.25] | 0.000000 | 0.000000 |
+---+-------+---------+--------------+--------------+----------+----------+
   Dim 0: Min=0.1499, Max=0.8194
   Dim 1: Min=0.0798, Max=0.8238
-> Z* (Ideal): [0.1499 0.0798]
-> Ref Point (HV): [1.4013 1.4062]
-> x (Feasible): [0.5]


In [16]:
pf_true = []
converge = 0
for r in test_rays:
    x, y, conv = solve_weighted_chebyshev(
        r_pref=r,
        z_ref=z_star, 
        dim_x=dim_x, 
        dim_y=dim_y,
        params=params)
    if conv:
        converge += 1
    pf_true.append(f(x))
print(f"Converge rate = {100*converge/len(test_rays)}%")

r=[1. 0.]
Iter  | x          | y vector        | Objective  | Active Idx
0     | 0.4990     | [0.59, 0.45] | 0.349078     | f_1
--> Hội tụ tại bước 961!

r=[0.9995 0.0321]
Iter  | x          | y vector        | Objective  | Active Idx
0     | 0.4990     | [0.59, 0.45] | 0.348899     | f_1
--> Hội tụ tại bước 961!

r=[0.9979 0.0641]
Iter  | x          | y vector        | Objective  | Active Idx
0     | 0.4990     | [0.59, 0.45] | 0.348363     | f_1
--> Hội tụ tại bước 962!

r=[0.9954 0.096 ]
Iter  | x          | y vector        | Objective  | Active Idx
0     | 0.4990     | [0.59, 0.45] | 0.347470     | f_1
--> Hội tụ tại bước 964!

r=[0.9918 0.1279]
Iter  | x          | y vector        | Objective  | Active Idx
0     | 0.4990     | [0.59, 0.45] | 0.346220     | f_1
--> Hội tụ tại bước 966!

r=[0.9872 0.1596]
Iter  | x          | y vector        | Objective  | Active Idx
0     | 0.4990     | [0.59, 0.45] | 0.344616     | f_1
--> Hội tụ tại bước 734!

r=[0.9816 0.1912]
Iter  | x         

# Giải xấp xỉ dynamic

## Tuning

In [17]:
from tuning_dynamic import tuning_dynamic

In [ ]:
best_params, df = tuning_dynamic(
    prob, z_star, x_feasible, pf_true, ref_point, test_rays
)

=== GRID SEARCH: ƯU TIÊN MED ===
Số lượng cấu hình: 96 | Số tia: 50


0it [00:00, ?it/s]

!! Max_iter. Delta: 0.029527, Gap C: 0.000000, Gap Q: 0.040122
!! Max_iter. Delta: 0.029512, Gap C: 0.000000, Gap Q: 0.040427
!! Max_iter. Delta: 0.029466, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.029391, Gap C: 0.000000, Gap Q: 0.000001
!! Max_iter. Delta: 0.005996, Gap C: 0.000000, Gap Q: 0.057900
!! Max_iter. Delta: 0.007375, Gap C: 0.000000, Gap Q: 0.035562
!! Max_iter. Delta: 0.008686, Gap C: 0.000000, Gap Q: 0.015001
!! Max_iter. Delta: 0.009931, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.011149, Gap C: 0.000000, Gap Q: 0.000010
!! Max_iter. Delta: 0.012345, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.028002, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.014544, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.015613, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.016618, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.026603, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.026156, Gap C: 0.000000, Gap Q: 0

1it [00:51, 51.75s/it]

!! Max_iter. Delta: 0.020671, Gap C: 0.000001, Gap Q: 0.035208
[00] MED: 8.297754e-03 | HV: 1.381720
!! Max_iter. Delta: 0.029527, Gap C: 0.000000, Gap Q: 0.036127
!! Max_iter. Delta: 0.029512, Gap C: 0.000000, Gap Q: 0.083792
!! Max_iter. Delta: 0.012812, Gap C: 0.000000, Gap Q: 0.085393
!! Max_iter. Delta: 0.029391, Gap C: 0.000000, Gap Q: 0.000001
!! Max_iter. Delta: 0.005996, Gap C: 0.000000, Gap Q: 0.057900
!! Max_iter. Delta: 0.007375, Gap C: 0.000000, Gap Q: 0.035562
!! Max_iter. Delta: 0.008686, Gap C: 0.000000, Gap Q: 0.015001
!! Max_iter. Delta: 0.009931, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.011149, Gap C: 0.000000, Gap Q: 0.000010
!! Max_iter. Delta: 0.012345, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.028002, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.014544, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.015613, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.016618, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delt

...

!! Max_iter. Delta: 0.049019, Gap C: 0.000001, Gap Q: 0.000000
[06] MED: 7.645104e-03 | HV: 1.379388
!! Max_iter. Delta: 0.059054, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.059024, Gap C: 0.000000, Gap Q: 0.042188
!! Max_iter. Delta: 0.037325, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.054580, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.000415, Gap C: 0.000000, Gap Q: 0.090286
!! Max_iter. Delta: 0.015028, Gap C: 0.000000, Gap Q: 0.083513
!! Max_iter. Delta: 0.017667, Gap C: 0.000000, Gap Q: 0.062136
!! Max_iter. Delta: 0.020249, Gap C: 0.000000, Gap Q: 0.042388
!! Max_iter. Delta: 0.022782, Gap C: 0.000000, Gap Q: 0.024042
!! Max_iter. Delta: 0.025143, Gap C: 0.000000, Gap Q: 0.006768
!! Max_iter. Delta: 0.027459, Gap C: 0.000000, Gap Q: 0.000005
!! Max_iter. Delta: 0.029734, Gap C: 0.000000, Gap Q: 0.000005
!! Max_iter. Delta: 0.054695, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.033942, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delt

8it [07:01, 53.17s/it]

!! Max_iter. Delta: 0.044395, Gap C: 0.000001, Gap Q: 0.000000
[07] MED: 9.892063e-03 | HV: 1.382534
!! Max_iter. Delta: 0.029527, Gap C: 0.000000, Gap Q: 0.040286
!! Max_iter. Delta: 0.029512, Gap C: 0.000000, Gap Q: 0.040613
!! Max_iter. Delta: 0.029466, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.029391, Gap C: 0.000000, Gap Q: 0.000001
!! Max_iter. Delta: 0.005996, Gap C: 0.000000, Gap Q: 0.057900
!! Max_iter. Delta: 0.007375, Gap C: 0.000000, Gap Q: 0.035562
!! Max_iter. Delta: 0.008686, Gap C: 0.000000, Gap Q: 0.015001
!! Max_iter. Delta: 0.009931, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.011149, Gap C: 0.000000, Gap Q: 0.000010
!! Max_iter. Delta: 0.012345, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.028002, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.014544, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.015613, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.016618, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delt

9it [07:53, 52.85s/it]

!! Max_iter. Delta: 0.020671, Gap C: 0.000001, Gap Q: 0.035208
[08] MED: 8.175701e-03 | HV: 1.381814
!! Max_iter. Delta: 0.029527, Gap C: 0.000000, Gap Q: 0.037875
!! Max_iter. Delta: 0.029512, Gap C: 0.000000, Gap Q: 0.083808
!! Max_iter. Delta: 0.013204, Gap C: 0.000000, Gap Q: 0.084805
!! Max_iter. Delta: 0.029391, Gap C: 0.000000, Gap Q: 0.000001
!! Max_iter. Delta: 0.005996, Gap C: 0.000000, Gap Q: 0.057900
!! Max_iter. Delta: 0.007375, Gap C: 0.000000, Gap Q: 0.035562
!! Max_iter. Delta: 0.008686, Gap C: 0.000000, Gap Q: 0.015001
!! Max_iter. Delta: 0.009931, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.011149, Gap C: 0.000000, Gap Q: 0.000010
!! Max_iter. Delta: 0.012345, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.028002, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.014544, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.015613, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.016618, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delt

## Best

In [ ]:
from Phase2_dynamic import optim_Scalarization
from evaluate import get_metrics

In [ ]:
pf_true_dynamic = []
for r in test_rays:
        x_final, _ = optim_Scalarization(
            prob=prob,
            x_feasible=x_feasible,  
            r=r, 
            z_star=z_star,
            # Các tham số tĩnh
            verbose=False,
            # --- CÁC THAM SỐ ĐANG TỐI ƯU ---
            max_iter=best_params['max_iter'],
            mu=best_params['mu'],
            init_params=best_params['init_params'],
            expo_alpha=best_params['expo_alpha'],
            expo_lambda=best_params['expo_lambda'],
            expo_beta=best_params['expo_beta'],
            expo_gamma=best_params['expo_gamma'],
        )
        pf_true_dynamic.append(prob.objective_func(x_final))
    
med, hv = get_metrics(pf_true_dynamic, pf_true, prob, ref_point)

In [ ]:
med, hv

## Viz

In [ ]:
from utils import visualize_pareto_front, generate_pareto_grid

In [ ]:
def non_c(x):
    return 1
pf_cloud, pf_targets = generate_pareto_grid(
    f_func=f, 
    c_funcs=[non_c], 
    q_plus_func=q_plus, 
    resolution=500
)

In [ ]:
visualize_pareto_front(
    pf_pred=np.array(pf_true_dynamic), 
    pf_cloud=pf_cloud,   
    pf_targets=np.array(pf_true),
    title="Dynamic approximate exact",
    figsize=(8, 6)
)

# Hypernet

In [ ]:
from training_hypernet import train_hypernet

In [ ]:
prob_ = Problem(
    f=[f1, f2], jac_f=[jacobian_f],
    dim_x=dim_x, dim_y=dim_y,
    proj_C=proj_C_handler.project,
    proj_Qplus=proj_Q_handler.project
)

## Tuning

In [ ]:
from tuning_hypernet import run_hypernet_tuning

In [ ]:
param_grid = {
    'lr': [1e-3, 5e-4],
    'num_epochs': [500, 1000, 2000],
    
    'num_partitions': [10],
    
    # Tham số thuật toán 2-A: Tăng dần penalty
    'beta_C_0': [10],
    'beta_Q_0': [5],
    'rho_C': [1.03], 
    'rho_Q': [1.01, 1.03],
    
    # Cố định Max để tránh grid quá lớn 
    'beta_C_max': [1000.0],
    'beta_Q_max': [100.0, 500.0, 1000.0]
}

In [ ]:
device = 'cpu'

In [ ]:
results = run_hypernet_tuning(
    prob=prob_,
    dim_x=dim_x,
    z_star=z_star,
    ref_point=ref_point,
    test_rays=test_rays,
    pf_true=pf_true,
    param_grid=param_grid,
    indicator="MED", 
    device=device,
    save_dir=f"model/{case}",
    train_func=train_hypernet
)

## Viz

In [ ]:
import pandas as pd

In [ ]:
df = pd.json_normalize(results, sep='_')
df.columns = df.columns.str.replace('params_', '')

In [ ]:
df.sort_values("med")

In [ ]:
df.to_csv(f"exp/{case}/{n_rays}.csv", index=False)

## Infer

In [ ]:
import os
import torch
import time
from hypernet_MLP import Hypernet_MLP
from hypernet_trans import Hypernet_trans
from utils import plot_trajectories

In [ ]:
def infer(model, test_rays=test_rays):
    model.eval()
    pf_pred = [] 
    tmp_err = [] 
    t1 = time.time()
    print(f"Evaluating on {len(test_rays)} rays...")

    for r in test_rays:   
        # --- Dự đoán với Hypernetwork ---
        ray_t = torch.from_numpy(r).float().unsqueeze(0)
        with torch.no_grad(): 
            output_x = model(ray_t).flatten() 
            vals = [func(output_x).item() for func in prob_.f]
            obj_pred = np.array(vals)

        pf_pred.append(obj_pred)
    t2 = time.time()
    pf_pred = np.array(pf_pred)
    print("-" * 30)
    print(f"Inference Time: {t2-t1:.4f}s")
    return pf_pred

In [ ]:
best_MLP = Hypernet_MLP(ray_hidden_dim=32, out_dim=dim_x, n_tasks=2)
best_trans = Hypernet_trans(ray_hidden_dim=32, out_dim=dim_x, n_tasks=2)
# 2. Đường dẫn file đã lưu
PATH_MLP = f"model/{case}/best_MLP_MED.pth"
PATH_trans = f"model/{case}/best_trans_MED.pth"
# 3. Load state dict
if os.path.exists(PATH_MLP):
    state_dict = torch.load(PATH_MLP, map_location=device)
    best_MLP.load_state_dict(state_dict)
    best_MLP.to(device)
    best_MLP.eval() 
    print("Successful MLP ✅")

if os.path.exists(PATH_trans):
    state_dict = torch.load(PATH_trans, map_location=device)
    best_trans.load_state_dict(state_dict)
    best_trans.to(device)
    best_trans.eval() 
    print("Successful Trans ✅")

In [ ]:
pf_pred = infer(best_trans)
med, hv = get_metrics(pf_pred, pf_true, prob_, ref_point)
med, hv

In [ ]:
plot_trajectories(obs_true=np.array(pf_true),obs_predict=np.array(pf_pred),obs_pareto_front=pf_cloud , figsize=(16, 8))

In [ ]:
pf_pred = infer(best_MLP)
med, hv = get_metrics(pf_pred, pf_true, prob, ref_point)
med, hv

In [ ]:
plot_trajectories(obs_true=np.array(pf_true),obs_predict=np.array(pf_pred),obs_pareto_front=pf_cloud , figsize=(16, 8))

# Summarize

In [ ]:
def run_benchmark(
    name,               # Tên hiển thị trong CSV (vd: "Hypernet_MLP", "Solver_BestParams")
    method_type,        # "model" hoặc "solver"
    prob,               # Đối tượng bài toán
    test_rays,          # Tập tia kiểm tra
    pf_true_gt,         # Ground Truth chuẩn để tính MED
    ref_point,          # Điểm tham chiếu tính HV
    # --- Optional cho Model ---
    model=None,
    # --- Optional cho Solver ---
    solver_func=None,
    x_feasible=None,
    z_star=None,
    solver_params=None, # Dictionary chứa {mu, max_iter, ...}
    # --- Settings ---
    csv_path="results_summary.csv",
    n_runs=None         # Số lần chạy để tính trung bình time (Mặc định: Model=10, Solver=1)
):
    """
    Hàm benchmark tổng hợp cho cả AI Model và Numerical Solver.
    """
    num_rays = len(test_rays)
    
    # 1. Xác định số lần chạy mặc định nếu không truyền vào
    if n_runs is None:
        n_runs = 10 if method_type == "model" else 1
    
    # 2. KIỂM TRA TRÙNG LẶP TRONG CSV
    if os.path.exists(csv_path):
        try:
            df_check = pd.read_csv(csv_path)
            is_exist = not df_check[(df_check['model'] == name) & 
                                    (df_check['num_rays'] == num_rays)].empty
            if is_exist:
                print(f"⚠️  [EXIST] Kết quả '{name}' ({num_rays} rays) đã có.")
                user_resp = input("   Chạy lại? (y/n): ").strip().lower()
                if user_resp != 'y':
                    return df_check
        except: pass

    print(f"\n🚀 BENCHMARKING: {name}")
    print(f"   Type: {method_type} | Rays: {num_rays} | Repeats: {n_runs}")

    # 3. CHUẨN BỊ (Warm-up cho Model)
    device = None
    if method_type == "model" and model is not None:
        model.eval()
        device = next(model.parameters()).device
        # Warm-up 1 pass
        dummy_ray = torch.from_numpy(test_rays[0]).float().unsqueeze(0).to(device)
        with torch.no_grad(): _ = model(dummy_ray)

    # 4. ĐO THỜI GIAN VÀ CHẠY
    total_time = 0
    pf_pred_final = []

    for i in range(n_runs):
        t1 = time.time()
        current_pf_vals = []

        # --- CASE A: DEEP LEARNING MODEL ---
        if method_type == "model":
            for r in test_rays:
                ray_t = torch.from_numpy(r).float().unsqueeze(0).to(device)
                with torch.no_grad():
                    output_x = model(ray_t).flatten()
                    x_cpu = output_x.cpu().numpy()
                    vals = [func(x_cpu).item() for func in prob.f]
                    current_pf_vals.append(vals)
        
        # --- CASE B: NUMERICAL SOLVER ---
        elif method_type == "solver":
            for r in test_rays:
                # Gọi solver với tham số unpack từ dict
                x_final, _ = solver_func(
                    prob=prob,
                    x_feasible=x_feasible,
                    r=r,
                    z_star=z_star,
                    verbose=False, # Tắt log chi tiết khi benchmark
                    max_iter=solver_params['max_iter'],
                    mu=solver_params['mu'],
                    init_params=solver_params['init_params'],
                    expo_alpha=solver_params['expo_alpha'],
                    expo_lambda=solver_params['expo_lambda'],
                    expo_beta=solver_params['expo_beta'],
                    expo_gamma=solver_params['expo_gamma']
                )
                current_pf_vals.append(prob.objective_func(x_final))
        
        t2 = time.time()
        run_time = t2 - t1
        total_time += run_time
        
        # In tiến độ nếu chạy Solver (vì lâu)
        if method_type == "solver":
            print(f"   Run {i+1}/{n_runs}: {run_time:.4f}s")
        
        # Lưu kết quả của lần chạy cuối cùng
        if i == n_runs - 1:
            pf_pred_final = np.array(current_pf_vals)

    avg_time = total_time / n_runs
    print(f"   ⏱️ Avg Time: {avg_time:.4f}s")

    # 5. TÍNH METRICS & LƯU
    med, hv = get_metrics(pf_pred_final, pf_true_gt, prob, ref_point)
    print(f"   📊 MED: {med:.5f} | HV: {hv:.5f}")

    new_row = {
        "model": name,
        "num_rays": num_rays,
        "med": med,
        "hv": hv,
        "time": avg_time
    }
    
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    else:
        df = pd.DataFrame([new_row])
        
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    df.to_csv(csv_path, index=False)
    print(f"✅ Saved to {csv_path}")
    return df

In [ ]:
csv_file = f"exp/{case}/results_summary.csv"

# 1. Benchmark Hypernetwork MLP
run_benchmark(
    name="Hypernet_MLP",
    method_type="model",
    prob=prob_,
    test_rays=test_rays,
    pf_true_gt=pf_true,      # Ground Truth chuẩn
    ref_point=ref_point,
    model=best_MLP,          # Truyền model vào
    csv_path=csv_file
)

# 2. Benchmark Hypernetwork Transformer
run_benchmark(
    name="Hypernet_Trans",
    method_type="model",
    prob=prob_,
    test_rays=test_rays,
    pf_true_gt=pf_true,
    ref_point=ref_point,
    model=best_trans,        # Truyền model vào
    csv_path=csv_file
)

# 3. Benchmark Numerical Solver (Với tham số tốt nhất)
run_benchmark(
    name="Numerical_Solver",
    method_type="solver",
    prob=prob,
    test_rays=test_rays,
    pf_true_gt=pf_true,
    ref_point=ref_point,
    solver_func=optim_Scalarization,  # Truyền hàm giải
    x_feasible=x_feasible,            # Input cần thiết cho solver
    z_star=z_star,                    # Input cần thiết cho solver
    solver_params=best_params,        # Dictionary chứa {mu, expo_alpha...} từ GridSearch
    csv_path=csv_file,
    n_runs=1                          # Chạy 1 lần vì lâu
)

In [ ]:
pd.read_csv(csv_file)